In [7]:
from h3 import h3
import pandas as pd
import numpy as np

In [8]:
import folium

from folium.plugins import FastMarkerCluster, MarkerCluster

In [18]:
taxi_data = pd.read_csv('5964-2008-02-02.csv', 
                        names=['taxi_id', 'start_ts', 'end_ts', 'hex_addr', 'count'], 
                        parse_dates=['start_ts', 'end_ts'])

In [19]:
taxi_data

,taxi_id,start_ts,end_ts,hex_addr,count
0,5964,2008-02-02 13:31:52+00:00,2008-02-02 13:31:52+00:00,8831aa55c3fffff,1
1,5964,2008-02-02 13:32:42+00:00,2008-02-02 13:35:54+00:00,8831aa55c1fffff,5
2,5964,2008-02-02 13:38:55+00:00,2008-02-02 13:41:56+00:00,8831aa5513fffff,4
3,5964,2008-02-02 13:36:54+00:00,2008-02-02 13:43:57+00:00,8831aa55cdfffff,6
4,5964,2008-02-02 13:44:57+00:00,2008-02-02 14:02:53+00:00,8831aa5427fffff,22
...,...,...,...,...,...
201,5964,2008-02-02 23:45:20+00:00,2008-02-02 23:45:20+00:00,8831aa470bfffff,1
202,5964,2008-02-02 23:46:20+00:00,2008-02-02 23:46:20+00:00,8831aa4701fffff,1
203,5964,2008-02-02 23:47:20+00:00,2008-02-02 23:47:46+00:00,8831aa4703fffff,2
204,5964,2008-02-02 23:48:21+00:00,2008-02-02 23:51:22+00:00,8831aa4707fffff,4


In [38]:
def plot_hexs(taxi_data):
    m = folium.Map(location=[(39.75 + 40.10)/2, (116.1 + 116.8)/2], 
                   zoom_start=11, 
                   prefer_canvas=True, 
                   tiles='stamentoner')
#                    tiles='cartodbpositron')
#                    tiles='Stamen Toner')

    for row_id, row in enumerate(taxi_data.iterrows()):
        row_data = row[1]        
        polygons = h3.h3_set_to_multi_polygon([row_data['hex_addr']], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        
        folium.PolyLine(locations=polyline,
                        weight=8,
                        color='crimson',
                        opacity=0.5,
                        popup='Taxi: {}, Count: {}'.format(row_data['taxi_id'], row_data['count']),
                        fill=True).add_to(m)

    return m

In [39]:
plot_hexs(taxi_data)

In [48]:
aggr_taxi_data = pd.read_csv('5964-2008-02-02-aggr.csv')

In [49]:
aggr_taxi_data

,hex_addr,seconds,weight
0,8831aa50bbfffff,4044,1.000000
1,8831aa5427fffff,2525,0.624382
2,8831aa55cdfffff,1449,0.358309
3,8831aa55c1fffff,1280,0.316518
4,8831aa425bfffff,937,0.231701
...,...,...,...
106,8831aa5627fffff,0,0.000000
107,8831aa55d3fffff,0,0.000000
108,8831aa50d3fffff,0,0.000000
109,8831aa5581fffff,0,0.000000


In [50]:
def plot_hexs_2(taxi_data):
    m = folium.Map(location=[(39.75 + 40.10)/2, (116.1 + 116.8)/2], 
                   zoom_start=11, 
                   prefer_canvas=True, 
                   tiles='stamentoner')
#                    tiles='cartodbpositron')
#                    tiles='Stamen Toner')

    for row_id, row in enumerate(taxi_data.iterrows()):
        row_data = row[1]
        if row_data['seconds'] == 0:
            continue
        
        polygons = h3.h3_set_to_multi_polygon([row_data['hex_addr']], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        
        folium.PolyLine(locations=polyline,
                        weight=8,
                        color='crimson',
                        opacity=row_data['weight'],
                        popup='Weight: {:2f}%'.format(row_data['weight'] * 100),
                        fill=True).add_to(m)

    return m

plot_hexs_2(aggr_taxi_data)